In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
import time

In [ ]:
script_start_time = time.time()

In [3]:
# df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")

In [4]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

## (0) Some minor data preprocessing

In [5]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)

In [6]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## (1) CO2 emission over time by continent

In [7]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=1750, end=2020, step=5, value=1850)
#year_slider

In [8]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['co2', 'co2_per_capita',],
    button_type='success'
)

In [9]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year','iso_code'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [10]:
#co2_pipeline

In [11]:
df_slice = df[df.country.isin(['World','Asia','Europe'])]
idf_slice = df_slice.interactive()

In [12]:
lst_country =list(set(df_slice.country))
country_checkbox = pn.widgets.ToggleGroup(name='Country',  options=lst_country, value=[lst_country[0]],button_type='success')
#display(country_checkbox)

In [13]:
if 0:    
    co2_slice_pipeline = (
        idf[
            (idf.year <= 1925) &
            (idf.country.isin(country_checkbox))
        ]
        .groupby(['country', 'year','iso_code'])['co2'].mean()
        .to_frame()
        .reset_index()
        .sort_values(by='year')  
        .reset_index(drop=True)
    )
    co2_slice_pipeline

In [14]:
if 0:
    co2_slice_plot = co2_slice_pipeline.hvplot(x = 'year', by=['iso_code','country'], y=['co2'],line_width=2, title="CO2 emission by continent")
    co2_slice_plot

In [15]:
co2_plot = co2_pipeline.hvplot(x = 'year', by=['iso_code','country'], y=yaxis_co2,line_width=2, title="CO2 emission by continent")
co2_plot

In [16]:
# lst_temp = list(np.linspace(0,9,10))
# lst_temp_slider = pn.widgets.DiscreteSlider(name='offset2', options=lst_temp,value=0)
# lst_temp_slider

In [17]:
bootstrap = pn.template.BootstrapTemplate(title='Bootstrap Template')

xs = np.linspace(0, np.pi)
freq = pn.widgets.FloatSlider(name="Frequency", start=0, end=10, value=2)
phase = pn.widgets.FloatSlider(name="Phase", start=0, end=np.pi)

@pn.depends(freq=freq, phase=phase)
def sine(freq, phase):
    return hv.Curve((xs, np.sin(xs*freq+phase))).opts(
        responsive=True, min_height=400)

@pn.depends(freq=freq, phase=phase)
def cosine(freq, phase):
    return hv.Curve((xs, np.cos(xs*freq+phase))).opts(
        responsive=True, min_height=400)

bootstrap.sidebar.append(freq)
bootstrap.sidebar.append(phase)

bootstrap.main.append(
    pn.Row(
        pn.Card(hv.DynamicMap(sine), title='Sine'),
        pn.Card(hv.DynamicMap(cosine), title='Cosine')
    )
)

## (2) Table - CO2 emission over time by continent 

In [18]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
co2_table

## (3) CO2 vs GDP scatterplot

In [19]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [20]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='co2', 
                                                                by='country', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
#co2_vs_gdp_scatterplot

## (4) Bar chart with CO2 sources by continent

In [21]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['coal_co2', 'oil_co2', 'gas_co2'], 
    button_type='success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [22]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_co2_source, 
                                                     title='CO2 source by continent')
#co2_source_bar_plot

## Creating Dashboard

In [23]:
# #Layout using Template
# template = pn.template.FastListTemplate(
#     title='World CO2 emission dashboard', 
#     sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
#              pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
#              pn.pane.PNG('./climate_day.png', sizing_mode='scale_both'),
#              pn.pane.Markdown("## Settings"),   
#              year_slider],
#     main=[pn.Row(pn.Column(yaxis_co2, 
#                            co2_plot.panel(width=700), margin=(0,25)), 
#                  co2_table.panel(width=500)), 
#           pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
#                  pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
#     accent_base_color="#88d8b0",
#     header_background="#88d8b0",
# )
# # template.show()
# template.servable();

In [24]:
#Layout using Template
timestamp_before_server = time.time()
template = pn.template.FastListTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."),              
             pn.pane.Markdown("## Settings"),   
             year_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           co2_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable()
end_time = time.time()
print("server setup time: %2.1f seconds."%(end_time - timestamp_before_server))
print("code exec. time: %2.1f seconds."%(time.time() - script_start_time))

server setup time: 0.7 seconds.
